<a href="https://colab.research.google.com/github/kotaxtech/pix2pixHD/blob/main/pix2pixhd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Githubリポジトリ：https://github.com/NVIDIA/pix2pixHD

## セットアップ


In [ ]:
# Colab上のGPUの確認．
!nvidia-smi

Sun Jan 16 14:02:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# MyDriveに移動し，workspaceディレクトリを作成．
# Githubから該当リポジトリをクローン．

from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/workspace
%cd /content/drive/MyDrive/workspace
!git clone https://github.com/NVIDIA/pix2pixHD.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 必要なライブラリ(dominate)をインストール．
# 評価指標となるFIDのライブラリは別途インストールする必要あり．
!pip install dominate
!pip install pytorch-fid

  Created wheel for pytorch-fid: filename=pytorch_fid-0.2.1-py3-none-any.whl size=14835 sha256=48838a4889d138904dfbb607fdcc95b6f5eeba987bcd697f3fe34753a4ea572b
  Stored in directory: /root/.cache/pip/wheels/24/ac/03/c5634775c8a64f702343ef5923278f8d3bb8c651debc4a6890
Successfully built pytorch-fid


## デモ


In [ ]:
# テストのデモ
#!./scripts/test_1024p.sh
%cd /content/drive/MyDrive/workspace/pix2pixHD
!python test.py --name label2city_1024p --netG local --ngf 32 --resize_or_crop none

/content/drive/MyDrive/workspace/Python/supervised /Pix2PixHD/pix2pixHD
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 35
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: label2city_1024p
nef: 16
netG: local
ngf: 32
niter_fix_global: 0
no_flip: False
no_instance: False
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: ./results/
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
------

## データセット

pix2pixHDディレクトリ直下にあるdatasetsディレクトリに以下のように自前のデータセットを配置する．原画像からラベル画像へ変換したい場合，trainA/testAには原画像，trainB/testBにはラベル画像を配置する．拡張子は特に記述はないが，pngかjpgの統一を推奨．
```
./datasets
  └ image2label
        ├ train_A
        │  ├ image0000.png
        │  ├ image0001.png
        │  │    ・
        │  │    ・
        │  │    ・
        │  └ imagexxxx.png
        ├ train_B
        ├ test_A
        └ test_B
```

## 学習(Image2Label)

Githubリポジトリより一部引用．Label2Imageの際，ラベル画像にグレースケール(インデックスカラー)画像でなくRGBから成るカラー画像を使用する場合，
 - 引数に**--label_nc 0**を指定すること．
 - datasetsディレクトリにて**train_label/train_img**から**train_A/train_B**にディレクトリ名を変更すること．

> If your input is not a label map, please just specify --label_nc 0 which will directly use the RGB colors as input. The folders should then be named train_A, train_B instead of train_label, train_img, where the goal is to translate images from A to B.

In [ ]:
!python train.py \
  --dataroot "./datasets/image2label/" \
  --name image2label \
  --loadSize 256 \
  --no_instance \
  --niter 25 \
  --niter_decay 25 \
  --label_nc 0 \
  #--continue_train 


# label2imageでグレースケール画像の場合…
# --label_nc 0 は指定しない．
'''
!python train.py \
  --dataroot "./datasets/label2image/" \
  --name label2image \
  --loadSize 256 \
  --no_instance \
  --niter 50 \
  --niter_decay 50 \
  #--continue_train 
'''

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/riceblast_label2img/
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 256
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: riceblast_label2img
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 50
niter_decay: 50
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: scale_width
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches:

## テスト(Image2Label)

In [ ]:
%cd /content/drive/MyDrive/workspace/pix2pixHD
!python test.py \
  --loadSize 256 \
  --dataroot "./datasets/image2label/" \
  --name image2label2 \
  --netG global \
  --no_instance \
  --label_nc 0 \
  --how_many 500 \
  --which_epoch latest

/content/drive/MyDrive/workspace/Python/supervised/Pix2PixHD/pix2pixHD
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/riceblast_label2img/
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 440
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 256
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: riceblast_label2img
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epo

## FIDの計算

In [ ]:
!mkdir /content/drive/MyDrive/workspace/pix2pixHD/results/image2label/test_latest/synthesized
!find ./results/image2label/test_latest/images -name "*synthesized*" | xargs -i cp {} "/content/drive/MyDrive/workspace/pix2pixHD/results/image2label/test_latest/synthesized"
!python -m pytorch_fid ./datasets/image2label/test_B/ ./results/image2label/test_latest/synthesized/

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:08<00:00, 11.5MB/s]
100% 2/2 [00:04<00:00,  2.08s/it]
100% 9/9 [00:05<00:00,  1.68it/s]
FID:  172.67819705348865


## 早くなるらしい…

In [ ]:
# 次回試す…
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./